# 奶牛疾病预测

首先，我们对已提供的 Excel 文件进行预处理，生成纯净的 CSV 文件，方便后续的数据处理。


In [258]:
import os
import pandas as pd
import numpy as np

DATA_FILES = [
    "../data/反刍数据0901-0915.xlsx",
    "../data/反刍数据0916-0930.xlsx",
    "../data/泌乳数据 2021_10_9 16-41-26.xlsx",
    "../data/活动数据20210901-20210915.xlsx",
    "../data/活动数据20210916-20210930.xlsx",
    "../data/疾病记录(2021年10月09日).xlsx",
]

# 加载并合并数据
df_chew1 = pd.read_excel(DATA_FILES[0], index_col=0)
df_chew1 = df_chew1.reset_index(drop=True)
df_chew2 = pd.read_excel(DATA_FILES[1], index_col=0)
df_chew2 = df_chew2.reset_index(drop=True)
df_chew = pd.concat([df_chew1, df_chew2], axis=0)
df_chew.reset_index(drop=True, inplace=True)

df_lactation = pd.read_excel(DATA_FILES[2], index_col=0)
df_lactation = df_lactation.reset_index(drop=True)

df_activity1 = pd.read_excel(DATA_FILES[3], index_col=0)
df_activity1 = df_activity1.reset_index(drop=True)
df_activity2 = pd.read_excel(DATA_FILES[4], index_col=0)
df_activity2 = df_activity2.reset_index(drop=True)
df_activity = pd.concat([df_activity1, df_activity2], axis=0)
df_activity.reset_index(drop=True, inplace=True)

df_disease = pd.read_excel(DATA_FILES[5])

d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default sty

## 1. 处理反刍数据

In [259]:
# 查看反刍数据
df_chew.info()
df_chew

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658607 entries, 0 to 658606
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   牛号           658607 non-null  int64         
 1   原始反刍数据       369211 non-null  float64       
 2   2小时反刍偏差      367334 non-null  float64       
 3   最近24小时总反刍时间  371435 non-null  float64       
 4   日反刍量         369324 non-null  float64       
 5   周平均反刍        369780 non-null  float64       
 6   加权反刍变化       366540 non-null  float64       
 7   日期           658559 non-null  datetime64[ns]
 8   时间           658559 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 45.2+ MB


,牛号,原始反刍数据,2小时反刍偏差,最近24小时总反刍时间,日反刍量,周平均反刍,加权反刍变化,日期,时间
0,6443,66.0,3.999092,588.0,586.0,577.0,-2.0,2021-09-01,0:00
1,6443,8.0,-10.385057,586.0,586.0,577.0,-3.0,2021-09-01,2:00
2,6443,57.0,-5.094848,559.0,586.0,577.0,-11.0,2021-09-01,4:00
3,6443,108.0,12.846859,587.0,586.0,577.0,-23.0,2021-09-01,6:00
4,6443,18.0,-4.668149,583.0,586.0,577.0,3.0,2021-09-01,8:00
...,...,...,...,...,...,...,...,...,...
658602,210566,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
658603,210567,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
658604,210568,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
658605,210569,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [260]:
# 将时间、日期合并转换为时间戳
df_chew["时间"] = df_chew["日期"].astype(str) + " " + df_chew["时间"].astype(str)
df_chew["时间"] = pd.to_datetime(df_chew["时间"], format="%Y%m%d %H:%M:%S", errors="coerce")
df_chew.drop(["日期"], axis=1, inplace=True)

df_chew

,牛号,原始反刍数据,2小时反刍偏差,最近24小时总反刍时间,日反刍量,周平均反刍,加权反刍变化,时间
0,6443,66.0,3.999092,588.0,586.0,577.0,-2.0,2021-09-01 00:00:00
1,6443,8.0,-10.385057,586.0,586.0,577.0,-3.0,2021-09-01 02:00:00
2,6443,57.0,-5.094848,559.0,586.0,577.0,-11.0,2021-09-01 04:00:00
3,6443,108.0,12.846859,587.0,586.0,577.0,-23.0,2021-09-01 06:00:00
4,6443,18.0,-4.668149,583.0,586.0,577.0,3.0,2021-09-01 08:00:00
...,...,...,...,...,...,...,...,...
658602,210566,NaN,NaN,NaN,NaN,NaN,NaN,NaT
658603,210567,NaN,NaN,NaN,NaN,NaN,NaN,NaT
658604,210568,NaN,NaN,NaN,NaN,NaN,NaN,NaT
658605,210569,NaN,NaN,NaN,NaN,NaN,NaN,NaT


重命名列名，并保存为 `chew.csv`。

| 字段名                 | 对应列名称          |
| ---------------------- | ------------------- |
| 牛号                   | `cow_id`            |
| 原始反刍数据           | `chew_data`         |
| 2 小时反刍偏差         | `chew_offset`       |
| 最近 24 小时总反刍时间 | `chew_time_daliy`   |
| 日反刍量               | `chew_number_daliy` |
| 周平均反刍             | `chew_time_weekly`  |
| 加权反刍变化           | `chew_delta`        |
| 时间                   | `datetime`          |


In [261]:
# 保存反刍数据
df_chew = df_chew.rename(
    columns={
        "牛号": "cow_id",
        "原始反刍数据": "chew_data",
        "2小时反刍偏差": "chew_offset",
        "最近24小时总反刍时间": "chew_time_daliy",
        "日反刍量": "chew_number_daliy",
        "周平均反刍": "chew_time_weekly",
        "加权反刍变化": "chew_delta",
        "时间": "datetime",
    }
)
df_chew.to_csv("../data/chew.csv")

## 2. 处理泌乳数据

In [262]:
# 查看泌乳数据
df_lactation.info()
df_lactation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248866 entries, 0 to 248865
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   牛号      248866 non-null  int64         
 1   日产量     210557 non-null  float64       
 2   日期      248866 non-null  datetime64[ns]
 3   日活动量    246547 non-null  float64       
 4   当前组别    248866 non-null  object        
 5   日反刍量    246086 non-null  float64       
 6   泌乳天数    248866 non-null  int64         
 7   泌乳期     248866 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 15.2+ MB


,牛号,日产量,日期,日活动量,当前组别,日反刍量,泌乳天数,泌乳期
0,6443,NaN,2021-01-01,382.0,泌乳22,542.0,370,4
1,6443,NaN,2021-01-02,372.0,泌乳22,502.0,371,4
2,6443,NaN,2021-01-03,379.0,泌乳22,574.0,372,4
3,6443,NaN,2021-01-04,405.0,泌乳22,528.0,373,4
4,6443,NaN,2021-01-05,406.0,泌乳22,568.0,374,4
...,...,...,...,...,...,...,...,...
248861,190580,6.16,2021-10-05,422.0,泌乳11,113.0,0,1
248862,190580,7.50,2021-10-06,607.0,泌乳11,245.0,1,1
248863,190580,14.75,2021-10-07,526.0,泌乳11,474.0,2,1
248864,190580,9.23,2021-10-08,525.0,泌乳11,632.0,3,1


In [263]:
# 日产量填充 0
df_lactation["日产量"].fillna(0, inplace=True)

# 去重
df_lactation.drop_duplicates(subset=["牛号", "日期"], inplace=True)

# 把反刍量小于 100 的数据去除
df_lactation = df_lactation[df_lactation["日反刍量"] > 100]

# 去除 2021-03-01（即生病统计之前）之前的数据
df_lactation = df_lactation[df_lactation["日期"] >= pd.to_datetime("2021-03-01")]

df_lactation.describe()

,牛号,日产量,日活动量,日反刍量,泌乳天数,泌乳期
count,198760.000000,198760.000000,198673.000000,198760.000000,198760.000000,198760.000000
mean,146233.895723,31.036623,492.901159,535.846986,188.083513,2.575377
std,64080.100188,16.653684,73.891227,84.100587,132.052582,1.428124
min,9.000000,0.000000,25.000000,101.000000,0.000000,1.000000
25%,160168.000000,22.893999,446.000000,496.000000,70.000000,1.000000
50%,170417.000000,34.380001,487.000000,545.000000,176.000000,2.000000
75%,180454.000000,42.796001,532.000000,589.000000,292.000000,4.000000
max,190580.000000,85.339996,1287.000000,1276.000000,1126.000000,6.000000


重命名列名，并保存为 `lactation.csv`。

| 字段名 | 对应列名称 |
| ------ | ---------- |
| 牛号   | `cow_id`     |
| 日产量 | `yield_daliy` |
| 日期   | `date`       |
| 日活动量 | `activity_daliy` |
| 当前组别 | `group` |
| 日反刍量 | `chew_daliy` |
| 泌乳天数 | `lactation_days` |
| 泌乳期 | `lactation_period` |


In [264]:
# 保存泌乳数据
df_lactation = df_lactation.rename(
    columns={
        "牛号": "cow_id",
        "日产量": "yield_daliy",
        "日期": "date",
        "日活动量": "activity_daliy",
        "当前组别": "group",
        "日反刍量": "chew_daliy",
        "泌乳天数": "lactation_days",
        "泌乳期": "lactation_period",
    }
)
df_lactation.to_csv("../data/lactation.csv")

## 3. 处理活动数据

In [265]:
# 查看活动数据
df_activity.info()
df_activity

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652966 entries, 0 to 652965
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   牛号       652966 non-null  int64         
 1   日期       652880 non-null  datetime64[ns]
 2   时间       652880 non-null  object        
 3   原始活动数据   368856 non-null  float64       
 4   2小时活动变化  366313 non-null  float64       
 5   日活动量     369192 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 29.9+ MB


,牛号,日期,时间,原始活动数据,2小时活动变化,日活动量
0,6443,2021-09-01,0:00,32.0,-3.317017,431.0
1,6443,2021-09-01,2:00,33.0,-2.560738,431.0
2,6443,2021-09-01,4:00,28.0,1.030928,431.0
3,6443,2021-09-01,6:00,35.0,-1.060236,431.0
4,6443,2021-09-01,8:00,38.0,-6.240377,431.0
...,...,...,...,...,...,...
652961,210586,NaT,NaN,NaN,NaN,NaN
652962,210587,NaT,NaN,NaN,NaN,NaN
652963,210588,NaT,NaN,NaN,NaN,NaN
652964,210589,NaT,NaN,NaN,NaN,NaN


In [266]:
# 将时间、日期合并转换为时间戳
df_activity["时间"] = df_activity["日期"].astype(str) + " " + df_activity["时间"].astype(str)
df_activity["时间"] = pd.to_datetime(df_activity["时间"], format="%Y%m%d %H:%M:%S", errors="coerce")
df_activity.drop(["日期"], axis=1, inplace=True)

df_activity

,牛号,时间,原始活动数据,2小时活动变化,日活动量
0,6443,2021-09-01 00:00:00,32.0,-3.317017,431.0
1,6443,2021-09-01 02:00:00,33.0,-2.560738,431.0
2,6443,2021-09-01 04:00:00,28.0,1.030928,431.0
3,6443,2021-09-01 06:00:00,35.0,-1.060236,431.0
4,6443,2021-09-01 08:00:00,38.0,-6.240377,431.0
...,...,...,...,...,...
652961,210586,NaT,NaN,NaN,NaN
652962,210587,NaT,NaN,NaN,NaN
652963,210588,NaT,NaN,NaN,NaN
652964,210589,NaT,NaN,NaN,NaN


重命名列，并保存为 `activity.csv`。

| 字段名         | 对应列名称       |
| -------------- | ---------------- |
| 牛号           | `cow_id`         |
| 时间           | `datetime`       |
| 原始活动数据   | `activity_data`  |
| 2 小时活动变化 | `activity_delta` |
| 日活动量       | `activity_daily` |


In [267]:
# 保存活动数据
df_activity = df_activity.rename(
    columns={
        "牛号": "cow_id",
        "时间": "datetime",
        "原始活动数据": "activity_data",
        "2小时活动变化": "activity_delta",
        "日活动量": "activity_daliy",
    }
)
df_activity.to_csv("../data/activity.csv")

## 4. 处理疾病记录

In [268]:
# 查看疾病记录
df_disease.info()

# 删除无用数据和隐私数据
df_disease.drop(["牧场名称", "疾病描述", "发病时牛群状态", "疾病揭发人", "兽医", "操作人"], axis=1, inplace=True)

# 值转换
df_disease["治疗结果"] = df_disease["治疗结果"].apply(lambda x: 1 if x.strip() == "治愈" else 0)
df_disease["发病时泌乳状态"] = df_disease["发病时泌乳状态"].apply(lambda x: 1 if x.strip() == "泌乳" else 0)
df_disease["左前"] = df_disease["左前"].apply(lambda x: 1 if str(x).strip() == "√" else 0)
df_disease["右前"] = df_disease["右前"].apply(lambda x: 1 if str(x).strip() == "√" else 0)
df_disease["左后"] = df_disease["左后"].apply(lambda x: 1 if str(x).strip() == "√" else 0)
df_disease["右后"] = df_disease["右后"].apply(lambda x: 1 if str(x).strip() == "√" else 0)

# 删除发病时泌乳天数
df_disease.drop(["发病时泌乳天数"], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   牧场名称     427 non-null    object 
 1   管理号      427 non-null    int64  
 2   牛舍编号     427 non-null    int64  
 3   牛舍名称     427 non-null    object 
 4   疾病名称     427 non-null    object 
 5   疾病分类     427 non-null    object 
 6   疾病描述     0 non-null      float64
 7   发病日期     427 non-null    object 
 8   治愈日期     427 non-null    object 
 9   治疗日期     427 non-null    object 
 10  治疗结果     427 non-null    object 
 11  发病时牛群状态  427 non-null    object 
 12  发病时泌乳状态  427 non-null    object 
 13  发病时在胎天数  427 non-null    int64  
 14  发病时泌乳天数  283 non-null    float64
 15  发病时月龄    427 non-null    float64
 16  发病时牛舍    427 non-null    object 
 17  发病时胎次    426 non-null    float64
 18  疾病揭发人    89 non-null     object 
 19  兽医       427 non-null    object 
 20  左前       164 non-null    object 
 21  右前       147 non

In [269]:
from datetime import timedelta

# 把发病日期，治愈日期转化为日期类型
df_disease["发病日期"] = pd.to_datetime(df_disease["发病日期"], errors="coerce")


def update_date(row, delta=5):
    """假设已经治愈的日期"""
    if row["治愈日期"] == "尚未治愈":
        row["治愈日期"] = row["发病日期"] + timedelta(days=delta)
    return row

df_disease = df_disease.apply(update_date, axis=1)
df_disease["治愈日期"] = pd.to_datetime(df_disease["治愈日期"], errors="coerce")

df_disease.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   管理号      427 non-null    int64         
 1   牛舍编号     427 non-null    int64         
 2   牛舍名称     427 non-null    object        
 3   疾病名称     427 non-null    object        
 4   疾病分类     427 non-null    object        
 5   发病日期     427 non-null    datetime64[ns]
 6   治愈日期     427 non-null    datetime64[ns]
 7   治疗日期     427 non-null    object        
 8   治疗结果     427 non-null    int64         
 9   发病时泌乳状态  427 non-null    int64         
 10  发病时在胎天数  427 non-null    int64         
 11  发病时月龄    427 non-null    float64       
 12  发病时牛舍    427 non-null    object        
 13  发病时胎次    426 non-null    float64       
 14  左前       427 non-null    int64         
 15  右前       427 non-null    int64         
 16  左后       427 non-null    int64         
 17  右后       427 non-null    int64     

重命名列，并保存为 `disease.csv`。

| 字段名         | 对应列名称           |
| -------------- | -------------------- |
| 管理号         | `management_number`  |
| 牛舍编号       | `cowshed_number`     |
| 牛舍名称       | `cowshed_name`       |
| 疾病名称       | `disease_name`       |
| 疾病分类       | `disease_cls`        |
| 发病日期       | `disease_date`       |
| 治愈日期       | `cure_date`          |
| 治疗日期       | `treatment_date`     |
| 治疗结果       | `treatment_outcomes` |
| 发病时泌乳状态 | `lactation_onset`    |
| 发病时在胎天数 | `pregnancy_days`     |
| 发病时泌乳天数 | `lactation_days`     |
| 发病时月龄     | `month_age`          |
| 发病时牛舍     | `cowshed_onset`      |
| 发病时胎次     | `birth_number`       |
| 左前           | `left_front`         |
| 右前           | `right_front`        |
| 左后           | `left_rear`          |
| 右后           | `right_rear`         |
| 操作时间       | `operation_time`     |


其他值设定：
- 泌乳状态：`1` 为泌乳，`0` 为干奶
- 治疗结果：`1` 为治愈，`0` 为其他


In [270]:
df_disease = df_disease.rename(
    columns={
        "管理号": "management_number",
        "牛舍编号": "cowshed_number",
        "牛舍名称": "cowshed_name",
        "疾病名称": "disease_name",
        "疾病分类": "disease_cls",
        "发病日期": "disease_date",
        "治愈日期": "cure_date",
        "治疗日期": "treatment_date",
        "治疗结果": "treatment_outcomes",
        "发病时泌乳状态": "lactation_onset",
        "发病时在胎天数": "pregnancy_days",
        "发病时泌乳天数": "lactation_days",
        "发病时月龄": "month_age",
        "发病时牛舍": "cowshed_onset",
        "发病时胎次": "birth_number",
        "左前": "left_front",
        "右前": "right_front",
        "左后": "left_rear",
        "右后": "right_rear",
        "操作时间": "operation_time",
    }
)  # type: ignore
df_disease.to_csv("../data/disease.csv")

## 5. 合并数据

In [271]:
# 合并泌乳和疾病数据
data = pd.merge(
    df_lactation,
    df_disease,
    left_on=["cow_id", "date"],
    right_on=["management_number", "disease_date"],
    how="left",
)

In [272]:
data_notnull_index = data[data["management_number"].notnull()].index
data_notnull_index

Int64Index([   784,    905,    997,   1720,   1731,   2645,   2730,   3080,
              3305,   3429,
            ...
            192800, 194354, 195173, 195179, 195767, 196022, 196120, 196195,
            197060, 197170],
           dtype='int64', length=394)

In [273]:
data["disease_status"] = np.nan

In [274]:
data_bak = data.copy()

In [275]:
data = data_bak.copy()

In [276]:
# 把疾病发现到治愈周期内都标注为疾病状态
for index in data_notnull_index:
    row = data.iloc[index]
    start = row["disease_date"]
    end = row["cure_date"]
    dif = pd.date_range(start, end)
    i = index + 1
    for d in dif:
        data["disease_status"].iloc[i] = row["disease_cls"]
        i += 1

        # 超过 5 天就认为样本不合格
        if i - index > 5:
            data["disease_status"].iloc[i] = "ERROR"

    i = index - 1
    # 将 14 天之前的数据标记为 ERROR
    for _ in range(14):
        if i < 0 or row["cow_id"] != data.iloc[i]["cow_id"]:
            break
        data["disease_status"].iloc[i] = "ERROR"
        i -= 1

    # 删除治愈后的 14 天数据
    i = index + len(dif) - 1
    for _ in range(14):
        if i >= len(data) or row["cow_id"] != data.iloc[i]["cow_id"]:
            break
        data["disease_status"].iloc[i] = "ERROR"
        i += 1

    # 向前数日期，数 1 天标记为疾病状态
    # i = index - 1
    # for _ in range(1):
    #     if row["cow_id"] != data.iloc[i]["cow_id"]:
    #         break
    #     data["disease_status"].iloc[i] = row["disease_cls"]
    #     i -= 1

    # 如果治疗大于 3 天，删除痊愈前 2 天数据
    # if len(dif) > 3:
    #     i = index + len(dif) - 2
    #     for _ in range(2):
    #         if i >= len(data) or row["cow_id"] != data.iloc[i]["cow_id"]:
    #             break
    #         data["disease_status"].iloc[i] = "ERROR"
    #         i += 1


# 删除 ERROR 标记的数据
data = data[data["disease_status"] != "ERROR"]

C:\Users\10069\AppData\Local\Temp\ipykernel_23024\3321689957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["disease_status"].iloc[i] = row["disease_cls"]


In [277]:
print(data_bak.shape)
print(data.shape)

(198761, 28)
(188572, 28)


In [278]:
# 保存数据
data.to_csv(os.path.join("../data/sample_test.csv"))